In [25]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from collections import Counter
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from xgboost import plot_tree
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import xgboost
import inspect
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
import warnings
import warnings
warnings.filterwarnings("ignore")

In [26]:
#load the dataset
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
df.head(3)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0


In [27]:
#Store the Id in a dataframe for submission
test_ids = test['Id']

In [28]:
#For the categorical columns ('Id' and 'EJ'), we use the 'fit_transform' method of the 'LabelEncoder' to transform the data.
encoder = LabelEncoder()
df['Id'] = encoder.fit_transform(df['Id']).astype(float)
test['Id'] = encoder.fit_transform(test['Id']).astype(float)

df['EJ'] = encoder.fit_transform(df['EJ']).astype(float)
test['EJ'] = encoder.fit_transform(test['EJ']).astype(float)

KNNImputer replaces missing values with the mean of the 'k' nearest neighbors in the feature space. 

In [29]:
# First, we create an instance of the 'KNNImputer' class.'n_neighbors=5', meaning the algorithm will use the 5 nearest neighbors to impute missing values.
imputer = KNNImputer(n_neighbors=5)

# Next, we extract the column names from the DataFrames for later use
df_col = df.columns
test_col = test.columns

# We use the 'fit_transform' method of the 'KNNImputer' to impute missing values in each DataFrame.
# The 'fit_transform' method returns a new array with the imputed values.
df_imputed = imputer.fit_transform(df)
test_imputed = imputer.fit_transform(test)

# Finally, we create new DataFrames using the imputed values and assign the original column names to them
df = pd.DataFrame(df_imputed, columns=df_col)
test = pd.DataFrame(test_imputed, columns=test_col)


In [30]:
#state the random to a value
state= 0

In [31]:
#Split the data for training removing the target and set the test size to 20%.

X = df.drop(columns = {"Class"})
y = df.pop('Class')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)

# Model: XGBoost

In [32]:
#Apply the model according to best parameters found with RandomSearch (see below) 

model = XGBClassifier(subsample= 0.9, 
                      min_child_weight = 2, 
                      max_depth= 2, 
                      n_estimators= 50,
                      gamma= 1, 
                      colsample_bytree= 0.8,
                      random_state= state)

In [33]:
#Fitting step of a machine learning model using scikit-learn. 
#This method is used to train a model on the provided training data 'X_train' and corresponding target labels 'y_train'.
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

In [34]:
#Check accuracy on the train set
y_train_pred_model = model.predict(X_train)
accuracy_score(y_train, y_train_pred_model)

0.9979716024340771

In [35]:
# testing accuracy
y_test_pred_model = model.predict(X_test)

accuracy_score(y_test, y_test_pred_model)

0.9112903225806451

Performs k-fold cross-validation on your machine learning model using scikit-learn's KFold and cross_val_score functions. This technique is used to assess the model's performance and to get an estimate of its accuracy on unseen data. 

In [36]:
Kfold = KFold(n_splits = 10)
results_K = cross_val_score(model, X, y, cv = Kfold)
print("KFold - Accuracy: {0}% ({1})%)".format(results_K.mean()*100, results_K.std()*100))

KFold - Accuracy: 93.19936541512428% (2.686383282476573)%)


Made the 'submission' DataFrame that contain the predicted probabilities for each class (class_0 and class_1) for each test sample. 

In [37]:
pred = model.predict_proba(test)[:, 1]

submission = pd.DataFrame({'Id': test_ids, 'class_0': 1 - pred, 'class_1': pred})
submission

,Id,class_0,class_1
0,00eed32682bb,0.584505,0.415495
1,010ebe33f668,0.584505,0.415495
2,02fa521e1838,0.584505,0.415495
3,040e15f562a2,0.584505,0.415495
4,046e85c7cc7f,0.584505,0.415495


# Find best parameters for XGBoost

In [451]:
train_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [452]:
Y = train_df['Class'].values
X = train_df.drop(['Class', 'Id'], axis=1)
test = test_df.drop(['Id'], axis=1)

In [453]:
X = pd.get_dummies(X, columns=['EJ'])

In [455]:
from sklearn.impute import KNNImputer

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Impute missing values
X_imputed = knn_imputer.fit_transform(X)
X_df = pd.DataFrame(X_imputed, columns=X.columns)

In [456]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [457]:
#give the model some parameters
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [458]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Randomized hyperparameter search using cross-validation to tune hyperparameters for an XGBoost model. It uses scikit-learn's RandomizedSearchCV with StratifiedKFold cross-validation.

'folds' is the number of folds you want to use for cross-validation.
In this case, 'folds = 3', meaning 3-fold cross-validation will be performed.

'param_comb' is the number of random combinations of hyperparameters to try during the search.

skf' is an instance of the 'StratifiedKFold' class, which is used to create stratified folds for cross-validation.
StratifiedKFold ensures that the class distribution is balanced in each fold, especially important for imbalanced datasets.

'n_splits' is the number of folds for cross-validation.

'shuffle' determines whether to shuffle the data before splitting into folds.

In [459]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 0)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

random_search.fit(X, Y)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[15:26:19] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7867f7172a40>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=600, n_jobs=None,
                                           nthread=1, num_parallel_tree=None,
                                           predictor=None, ...),
                   n_iter=5, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=1001, scoring='roc_auc', verbose=3)

In [460]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


 All results:
{'mean_fit_time': array([2.17979725, 2.27372114, 1.60088952, 1.73772089, 2.35970966]), 'std_fit_time': array([0.03073489, 0.04302759, 0.0415487 , 0.01940789, 0.43878059]), 'mean_score_time': array([0.01357802, 0.0130651 , 0.01271033, 0.0119319 , 0.01011475]), 'std_score_time': array([0.00211679, 0.0004664 , 0.00024398, 0.00034908, 0.00063248]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dt